## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-04-23-49-22 +0000,nyt,Supreme Court Clears Way for Texas to Use Repu...,https://www.nytimes.com/2025/12/04/us/politics...
1,2025-12-04-23-48-18 +0000,bbc,Suspect arrested in pipe bomb incident on eve ...,https://www.bbc.com/news/articles/c62d8v6jpwzo...
2,2025-12-04-23-45-34 +0000,nypost,Cup o’ woe: 12 Starbucks strikers arrested out...,https://nypost.com/2025/12/04/us-news/12-starb...
3,2025-12-04-23-45-06 +0000,nyt,Tim Walz Calls Trump’s Tirade Against Somalis ...,https://www.nytimes.com/2025/12/04/us/tim-walz...
4,2025-12-04-23-43-03 +0000,cbc,Liberal bill to strip military from handling s...,https://www.cbc.ca/news/politics/proposed-legi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2375/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
28,trump,45
51,new,18
382,ukraine,14
449,putin,13
97,was,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
134,2025-12-04-19-19-46 +0000,nypost,Accused DC pipe-bomber Brian Cole Jr.’s nabbed...,https://nypost.com/2025/12/04/us-news/accused-...,108
15,2025-12-04-23-23-00 +0000,wsj,The Trump administration intends to formally s...,https://www.wsj.com/politics/policy/white-hous...,101
290,2025-12-04-10-00-00 +0000,wsj,Sen. Elizabeth Warren and other Democrats are ...,https://www.wsj.com/politics/policy/democrats-...,95
353,2025-12-04-00-05-45 +0000,nyt,Trump Faces Choices on Russia-Ukraine Talks as...,https://www.nytimes.com/2025/12/03/us/politics...,94
100,2025-12-04-21-11-38 +0000,nypost,Trump boasts heads of Rwanda and Congo will be...,https://nypost.com/2025/12/04/us-news/trump-bo...,86


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
134,108,2025-12-04-19-19-46 +0000,nypost,Accused DC pipe-bomber Brian Cole Jr.’s nabbed...,https://nypost.com/2025/12/04/us-news/accused-...
357,71,2025-12-04-00-00-00 +0000,wsj,Defense Secretary Pete Hegseth shocked officia...,https://www.wsj.com/politics/national-security...
61,61,2025-12-04-22-20-21 +0000,bbc,Putin says Russia will take Donbas by force or...,https://www.bbc.com/news/articles/cgexgwr1wrro...
15,56,2025-12-04-23-23-00 +0000,wsj,The Trump administration intends to formally s...,https://www.wsj.com/politics/policy/white-hous...
262,47,2025-12-04-11-02-24 +0000,bbc,Israel says Palestinian groups returned body o...,https://www.bbc.com/news/articles/cd6x2j4g3z7o...
16,43,2025-12-04-23-20-00 +0000,wsj,Lawmakers were briefed by Adm. Frank “Mitch” B...,https://www.wsj.com/politics/national-security...
290,38,2025-12-04-10-00-00 +0000,wsj,Sen. Elizabeth Warren and other Democrats are ...,https://www.wsj.com/politics/policy/democrats-...
246,35,2025-12-04-12-30-00 +0000,latimes,California hammered as national job cuts jump ...,https://www.latimes.com/business/story/2025-12...
153,34,2025-12-04-18-17-00 +0000,wsj,Olivia Nuzzi and Ryan Lizza Went From a $1 Mil...,https://www.wsj.com/business/media/olivia-nuzz...
8,32,2025-12-04-23-30-36 +0000,nypost,Supreme Court allows Texas to use newly drawn ...,https://nypost.com/2025/12/04/us-news/supreme-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
